In [4]:
import sys
sys.path.append('C:/Users/ilobach/Documents/SRW/SRW-master/env/work/srw_python')
sys.path.append('C:/Users/lobac_000/Documents/UHCICAGO/Thesis/SRW/SRW-master/env/work/srw_python')
sys.path.append('G:/My Drive/UHCICAGO/Git/figure_arrange_scripts')
import figure_arrange_scripts as ff
sec_f = 24

from srwlib import *
from uti_plot import *
from scipy.ndimage.interpolation import zoom
import tables

import os

import numpy as np
import pandas as pd
from itertools import islice


from scipy.interpolate import interp1d
from scipy.interpolate import interp2d


def align_yaxis(ax1, v1, ax2, v2):
    """adjust ax2 ylimit so that v2 in ax2 is aligned to v1 in ax1"""
    _, y1 = ax1.transData.transform((0, v1))
    _, y2 = ax2.transData.transform((0, v2))
    inv = ax2.transData.inverted()
    _, dy = inv.transform((0, 0)) - inv.transform((0, y1-y2))
    miny, maxy = ax2.get_ylim()
    ax2.set_ylim(miny+dy, maxy+dy)
    
    
from scipy.misc import electrocardiogram
from scipy.signal import find_peaks,peak_widths

#import tables
import os.path as path

from scipy.interpolate import CubicSpline

In [5]:
#fundamental constants
qe = 1.6021766208e-19
c = 299792458
mu_0 = 4*np.pi*1e-7
eps_0 = 1/(c**2*mu_0)
me = 9.10938356e-31
mp = 1.6726219e-27
h = 6.62607e-34

In [6]:
#**********************Undulator constants:
numPer = 10 #6#Number of ID Periods (without counting for terminations; semi-integer => symmetric vs long. pos.; to allow harmonics to be symmetric or anti-symmetric?)
undPer = 0.055 #0.1106 #Period Length [m]
phBy = 0 #Initial Phase of the Vertical field component
Kund = 1.0 #Kund = qe*By*undPer/(2*np.pi*me*c)
Lund = numPer*undPer#1.7675
By = Kund*(2*np.pi*me*c)/(qe*undPer)#0.1514815555529875


part = SRWLParticle()
part.x = 0.0 #Initial Transverse Coordinates (initial Longitudinal Coordinate will be defined later on) [m]
part.y = 0.0
#part.z = -0.5*(numPer + 4)*undPer #Initial Longitudinal Coordinate (set 2 periods before pickup)
part.z = -0.5*(numPer + 4)*undPer #- 1.3981378
part.xp = 0 #Initial Transverse Velocities
part.yp = 0
part.gamma = 0.100/0.51099890221e-03 #Relative Energy
part.relE0 = 1 #Electron Rest Mass
part.nq = -1 #Electron Charge


npTraj = 20000 #Number of Points for Trajectory calculation
dispBy = array('d', [0]*npTraj)
partTraj = SRWLPrtTrj(_arBy=dispBy) #you can consider it as a part of allocation, values of B field are still zero here
partTraj.partInitCond = part
partTraj.allocate(npTraj, False)
partTraj.ctStart = 0.0 #Start Time for the calculation
#partTraj.ctEnd = fmult*undPer*(numPer+4) #End Time ~1.2166
#partTraj.ctEnd = undPer*(numPer+4)/2+1.3981378+3*zdcent_f
partTraj.ctEnd = undPer*(numPer+4) #+1.#+3*zdcent_f

lmb_0 = undPer/(2*part.gamma**2)*(1 + Kund**2/2.0)

In [7]:
#***********Magnetic Field
sBy = 1 #Symmetry of the Vertical field component vs Longitudinal position
xcID = 0 #Transverse Coordinates of Undulator Center [m]
ycID = 0
zcID = 0 #Longitudinal Coordinate of Undulator Center [m]
und = SRWLMagFldU([SRWLMagFldH(1, 'v', By, phBy, sBy, 1)], undPer, numPer) #Planar Undulator
magFldCnt = SRWLMagFldC([und], array('d', [xcID]), array('d', [ycID]), array('d', [zcID])) #Container of all Field Elements

In [8]:
partTraj = srwl.CalcPartTraj(partTraj, magFldCnt, [1])# why was the last argument 0 and not [1] or something like this

arX = np.asarray(partTraj.arX)
arZ = np.asarray(partTraj.arZ)
arXp = np.asarray(partTraj.arXp)
arZp = np.asarray(partTraj.arZp)

ct_list = np.linspace(partTraj.ctStart, partTraj.ctEnd, partTraj.np)

In [9]:
#***********Electron Beam
f_ring = 7.5e6
Iavg_se = qe*f_ring

elecBeam = SRWLPartBeam()
elecBeam.Iavg = Iavg_se #Average Current [A]
elecBeam.partStatMom1 = part

Iavg_se

1.2016324656000001e-12

In [10]:
n_l = 80
l_min = 0.85
l_max = 1.3
l_list = np.linspace(l_min, l_max, n_l)  # um
x_max = 0.0254  # m
y_max = x_max/np.sqrt(2)  # m
n_x = 512
n_y = int(n_x/np.sqrt(2))

In [11]:
#***********Precision Parameters for SR calculation
meth = 1 #SR calculation method: 0- "manual", 1- "auto-undulator", 2- "auto-wiggler"
relPrec = 0.01 #relative precision
zStartInteg = 0 #longitudinal position to start integration (effective if < zEndInteg)
zEndInteg = 0 #longitudinal position to finish integration (effective if > zStartInteg)
useTermin = 1 #Use "terminating terms" (i.e. asymptotic expansions at zStartInteg and zEndInteg) or not (1 or 0 respectively)
sampFactNxNyForProp = 0 #sampling factor for adjusting nx, ny (effective if > 0)
arPrecPar = [meth, relPrec, zStartInteg, zEndInteg, npTraj, useTermin, sampFactNxNyForProp]
xs = -x_max
xf = x_max
ys = -y_max
yf = y_max
zobs = 3.5
xbins = 2*n_x
ybins = 2*n_y
Ex_3D = np.zeros(shape=(n_l, ybins, xbins), dtype=complex)
Ey_3D = np.zeros(shape=(n_l, ybins, xbins), dtype=complex)

In [12]:
def to_2D(values_1D):
    np_E = np.asarray(values_1D)
    np_E_Real = np_E[::2].reshape((ybins,xbins))
    np_E_Imag = np_E[1::2].reshape((ybins,xbins))
    return np_E_Real+1j*np_E_Imag

In [13]:
for k, l in enumerate(l_list):
    print('Working on {} out of {}'.format(k+1, len(l_list)),' ... ', end='')
    wfr2 = SRWLWfr() #For intensity distribution at fixed photon energy
    wfr2.allocate(1, xbins, ybins) #Numbers of points vs Photon Energy, Horizontal and Vertical Positions
    wfr2.mesh.zStart = zobs #Longitudinal Position [m] at which SR has to be calculated
    wfr2.mesh.xStart = xs #Initial Horizontal Position [m]
    wfr2.mesh.xFin = xf #Final Horizontal Position [m]
    wfr2.mesh.yStart = ys #Initial Vertical Position [m]
    wfr2.mesh.yFin = yf #Final Vertical Position [m]
    wfr2.partBeam = elecBeam
    pe = 1.24/l
    wfr2.mesh.eStart = pe #Initial Photon Energy [eV]
    wfr2.mesh.eFin =  pe #Final Photon Energy [eV]
    srwl.CalcElecFieldSR(wfr2, 0, magFldCnt, arPrecPar)
    print('done', wfr2.mesh.nx)
    Ex_3D[k,:,:] = to_2D(wfr2.arEx)
    Ey_3D[k,:,:] = to_2D(wfr2.arEy)

Working on 1 out of 80  ... done 1024
Working on 2 out of 80  ... done 1024
Working on 3 out of 80  ...done 1024
Working on 4 out of 80  ... done 1024
Working on 5 out of 80  ...done 1024
Working on 6 out of 80  ...done 1024
Working on 7 out of 80  ...done 1024
Working on 8 out of 80  ...done 1024
Working on 9 out of 80  ...done 1024
Working on 10 out of 80  ... done 1024
Working on 11 out of 80  ...done 1024
Working on 12 out of 80  ...done 1024
Working on 13 out of 80  ...done 1024
Working on 14 out of 80  ...done 1024
Working on 15 out of 80  ...done 1024
Working on 16 out of 80  ...done 1024
Working on 17 out of 80  ... done 1024
Working on 18 out of 80  ...done 1024
Working on 19 out of 80  ...done 1024
Working on 20 out of 80  ...done 1024
Working on 21 out of 80  ... done 1024
Working on 22 out of 80  ... done 1024
Working on 23 out of 80  ...done 1024
Working on 24 out of 80  ... done 1024
Working on 25 out of 80  ...done 1024
Working on 26 out of 80  ...done 1024
Working on 27

In [14]:
dx = (xf-xs)/(xbins-1)
dy = (yf-ys)/(ybins-1)
dl = (l_list[-1]-l_list[0])/(n_l-1)
coef = zobs*np.sqrt(1e9*qe/Iavg_se/l_list)
Ex_3D = coef[:, None, None]*Ex_3D
Ey_3D = coef[:, None, None]*Ey_3D

In [20]:
dx/zobs*dy/zobs*dl*np.sum(np.absolute(Ex_3D)**2)

0.024174194399202566

In [21]:
dx/zobs*dy/zobs*dl*np.sum(np.absolute(Ey_3D)**2)

0.0012995719645260953

In [22]:
dx/zobs*dy/zobs*dl*np.sum(np.absolute(Ex_3D)**2+np.absolute(Ey_3D)**2)

0.025473766363728584

In [18]:
np.save('Ex_3D.npy', Ex_3D)

In [19]:
np.save('Ey_3D.npy', Ey_3D)